In [54]:
import pandas as pd

In [55]:
import os 
import ast

def find_repo_root(start_path):
    """
    useful general function for finding the (first, closest) repo root so github file paths work the same on different machines 
    """
    current_path = os.path.abspath(start_path)
    
    while True:
        # Check for the existence of the .git directory or other indicators
        if os.path.isdir(os.path.join(current_path, '.git')) or \
           os.path.isfile(os.path.join(current_path, 'README.md')):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        
        # Stop if we reach the root directory
        if parent_path == current_path:
            break
        
        current_path = parent_path

    return None  # Return None if not found

root = find_repo_root(os.getcwd())


def fk_apply_literal(x):
    try:
        return ast.literal_eval(x)
    except: 
        return None




In [56]:
df = pd.read_csv(f"{root}/Data/movie_n=8_comments.csv")
df['comments'][0]

'[{\'stars\': \'★★★★\', \'review\': \'Yo no entendí bien la película, Perón sabía que López Rega le tendía una trampa?\', \'date\': \'09 Nov 2020\'}, {\'stars\': \'★★★★½\', \'review\': \'Me pareció excelente. Impecable en términos históricos y con una forma de contar la historia bastante original y que funciona muy bien.Más que un Perón épico exiliado, está el Perón humano, que sufre la vil proscripción y sigue llorando por Eva. Increíblemente amena, maneja 18 años de historia muy bien explicados, sumado a la ambientación que está bien y la actuación de Víctor Laplace que está igualito al General.Tal vez pesa mucho que se haya hecho en pleno…\', \'date\': \'17 Jul 2020\'}, {\'stars\': \'★★\', \'review\': \'This review may contain spoilers.I can handle the truth.Esta película cuando la vi por primera vez me gustó mucho, pero volviéndola a ver siento que tiene muchísimos problemas. Es una película que no se calla nunca, todas las escenas tienen diálogos, es "Perón pensando en voz alta" y

In [57]:
def extract_reviews(reviews_str):
    try:
        # Convert the string representation of the list to an actual list
        reviews_list = ast.literal_eval(reviews_str)
        # Extract 'review' from each dictionary
        return [review['review'] for review in reviews_list]
    except (ValueError, SyntaxError):
        return []
df['reviews_extracted'] = df['comments'].apply(extract_reviews)
df['reviews_mash'] = df['reviews_extracted'].apply(lambda x: '\n'.join(x))
print(df['reviews_mash'][2])
print(df['Movie'][2])

Matt Damon knocks it out of the park.
it's better than anomalisa
Chris warned us about Cats but we didn't listen.
I choose to view this as a documentary.
Matt damon
The puking scene is comedy gold.
don't think this is quite as funny as i once did (and the "eww, gay stuff" vibe is even less welcome now), but i'd definitely forgotten about its visual virtuosity and how well it absorbed the motifs of American action movies. the best gags are simple nuts-and-bolts jokes about the limitations of the marionettes themselves, like when an elaborate fight is just a bunch of wriggling dolls, or when they have to make one look like a staggering drunk. well, that and the sex scene.
THE POLITICAL SATIRE: On the one hand, you've got the destructive excesses of American foreign policy. On the other, you've got some Hollywood libs who are a bit annoying. I think I know what Matt and Trey hate more.THE ACTION-MOVIE SATIRE: Replicates the cliches of bad Hollywood action movies so faithfully that it's...

In [58]:

df['Movie'] = df['Movie'].apply(lambda x : fk_apply_literal(x))
df['genres'] = df['Movie'].apply(lambda x: x.get('genres'))

In [65]:
df['genres'].value_counts()

genres
[Drama]                        5
[Drama, Comedy]                3
[Comedy]                       3
[Horror]                       3
[Science Fiction, Horror]      2
                              ..
[Thriller, Drama, Crime]       1
[Drama, Crime, Comedy]         1
[Comedy, Romance, Drama]       1
[Western, Adventure, Drama]    1
[Thriller, Drama, Action]      1
Name: count, Length: 62, dtype: int64